In [ ]:
from transformers import pipeline
import streamlit as st
from bs4 import BeautifulSoup
import json
import requests
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.agents.agent import AgentExecutor
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory

from langchain.tools import BaseTool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chains import LLMChain, ConversationChain
from typing import Union

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,question_answering
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.llms import LlamaCpp

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
        try:
            # this will work IF the text is a valid JSON with action and action_input
            print(text)
            response = parse_json_markdown(text)
            action, action_input = response["action"], response["action_input"]
            if action == "Final Answer":
                # this means the agent is finished so we call AgentFinish
                return AgentFinish({"output": action_input}, text)
            else:
                # otherwise the agent wants to use an action, so we call AgentAction
                return AgentAction(action, action_input, text)
        except Exception:
            # sometimes the agent will return a string that is not a valid JSON
            # often this happens when the agent is finished
            # so we just return the text as the output
            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

parser = OutputParser()

In [ ]:
# Load
repo_path = r"E:\Projects\deOlival2023\lastreviAGOST\de_olival_python\de_olival_python\home"
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()
len(documents)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

In [ ]:
texts[0]

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
"""
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
model.query_instruction = "为这个句子生成表示以用于检索相关文章："

"""



In [ ]:
db = Chroma.from_documents(texts, HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
))
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
PATH = r"H:\oogabooga\text-generation-webui-1.6.1\models\codellama-13b-instruct.Q4_K_M\codellama-13b-instruct.Q4_K_M.gguf"
llm = LlamaCpp(model_path=PATH, 
                n_gpu_layers=43,
                n_batch=512,
                n_ctx=5000,
                f16_kv=True,#thing in case
                callback_manager=callback_manager,
                verbose=True)


In [ ]:
llm("How can i make a model in django")

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

#Prompt working
system_prompt = """You are a helpful assistant, you will use the provided context to answer questions.
Read the given context before answering questions and think step by step. If you can not answer a user question based on
the provided context, inform the user. Do not use any other information for answerin user"""

instruction = """
Context : {context}
User: {question}"""

#simple string join function
def prompt_format(instruction= instruction, system_prompt= system_prompt ):
    SYSTEM_PROMPT = B_SYS + system_prompt + E_SYS
    prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [ ]:
#Prompt by default, not working
template = """Use the following pieces of context to answer the question at the end.
If you don`t know the answer, just say that you don`t know, don`t try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""

In [ ]:

template = prompt_format()

#Question
question = "Describe the templates purpouse"
docs = retriever.get_relevant_documents(question)

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context","question"],
    template=template,
)




In [ ]:
QA_CHAIN_PROMPT

In [ ]:
#Chain
chain = load_qa_chain(llm, chain_type="stuff",prompt = QA_CHAIN_PROMPT )


In [ ]:
#Run
chain({"input_documents": docs, "question": question},return_only_outputs = True)